In [ ]:
import ssl 

# Deshabilitar la verificación SSL
ssl._create_default_https_context = ssl._create_unverified_context

from Bio import Entrez, SeqIO 

# Obtener el genoma de E. coli MG1655
def fetch_ecoli_genome(): 
    '''
    Esta función se utiliza para obtener los datos del genoma de E. coli MG1655 desde la base de datos NCBI.
    Recupera los datos mediante el ID de GenBank especificado ("U00096.3")
    '''
    
    # Email necesario para las consultas a la API de NCBI (similar a una clave de API)
    Entrez.email = "facundomrb@gmail.com"  
    # Realiza una petición a la API de NCBI para obtener datos genómicos
    # db="nucleotide": Base de datos de secuencias de nucleótidos (ADN/ARN)
    # id="U00096.3": Identificador único del genoma de E. coli MG1655
    # rettype="gb", retmode="text": Formato de retorno (GenBank como texto)
    handle = Entrez.efetch(db="nucleotide", id="U00096.3", rettype="gb", retmode="text") 
    # Parsea la respuesta en formato GenBank (similar a parsear un JSON o XML)
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

# Función para encontrar la posición de un gen específico en el genoma
# Equivalente a buscar un registro específico en una base de datos
def find_gene_position(record, gene_name):
    # Itera a través de todas las características (features) del registro genómico
    for feature in record.features:
        # CDS = Coding Sequence (secuencia codificante, que representa un gen que codifica una proteína)
        # Verifica si es una secuencia codificante y tiene un calificador de "gene"
        if feature.type == "CDS" and "gene" in feature.qualifiers:
            # Comprueba si el nombre del gen coincide con el buscado
            if feature.qualifiers["gene"][0] == gene_name:
                # Devuelve las posiciones de inicio y fin del gen en el genoma (como índices en un string)
                return feature.location.start, feature.location.end
    # Si no se encuentra el gen, devuelve None
    return None  

# Ejecuta la función para obtener el genoma completo de E. coli
record = fetch_ecoli_genome()


In [2]:
## Este módulo se utiliza para extraer genes específicos del genoma
# Lista de genes que queremos extraer (similar a una lista de IDs en una base de datos)
genes_to_extract =['gpmA', 'fbaA', 'pykF', 'pfkB', 'pykA', 'gpmM', 'pfkA', 'fbaB']
# Seleccionamos un gen específico para este ejemplo
gene_name = "gpmA"
# Buscamos la posición del gen en el genoma
gene_position = find_gene_position(record, gene_name)

if gene_position is not None:
    # Extrae la secuencia del gen, incluyendo 100 bases antes del inicio del gen
    # Esto es como extraer un substring, pero con datos biológicos
    # .lower() convierte todas las letras a minúsculas (convención en bioinformática)
    gene_seq = record.seq[gene_position[0]-100:gene_position[1]].lower()
    print(f"Found gene {gene_name} at positions {gene_position[0]} to {gene_position[1]}")
    # .upper() convierte a mayúsculas para visualización
    print(f"Gene sequence:\n{gene_seq.upper()}")
    print('Length: ',len(gene_seq.upper()))
    
    # Extrae una porción más pequeña del gen (100 bases antes y 300 después del inicio)
    # Esto es útil para análisis más específicos de regiones del gen
    print(f"Found gene {gene_name} -100:300 position:")
    gene_truncate = record.seq[gene_position[0]-100:gene_position[0]+300].lower()
    print(gene_truncate.upper())
    print('Length: ',len(gene_truncate.upper()))

else:
    print(f"Gene {gene_name} not found in the genome.") 

Found gene gpmA at positions 786842 to 787595
Gene sequence:
GATAAGTCGCGCAGCGTCGCATCAGGCAATGTGCTCCATTGTTAGCAACAAAAAAGCCGACTCACTTGCAGTCGGCTTTCTCATTTTAAACGAATGACGTTTACTTCGCTTTACCCTGGTTTGCAACCGCCGCTGCTTTCGCTGCGATCTCGTCAGCATTACCCAGATAATAGCGTTTCAGCGGTTTGAAATTCTCGTCGAACTCATACACCAGCGGCACGCCAGTCGGGATATTAAGCTCAAGAATCTCTTCTTCGCTCATGTTATCAAGATATTTCACCAGCGCACGTAAAGAGTTACCGTGTGCAGCGATGATCACGCGCTCACCGCTCTTCATACGCGGCAGAATAGTTTCATTCCAGTAAGGGATCACGCGGTCAATGGTCAGCGCCAGGCTTTCCGTCAGCGGCAGTTCTTTCTCGCTCAGTTTCGCGTAACGCGGATCGTGACCCGGATAACGCTCATCATCTTTAGTCAGTTCCGGCGGAGTCACTGCAAAACCACGACGCCACTGTTTCACCTGCTCGTCGCCATACTTTTCAGCAGTTTCCGCTTTGTTCAGACCCTGCAACGCACCGTAGTGACGTTCGTTCAGTTTCCAGGATTTCTCAACGGGCAGCCATGCCTGATCCAGTTCGTCCAGCACATTCCACAGGGTATGGATAGCGCGTTTCAGCACAGAAGTGTAAGCAAAGTCAAAGCTGTAACCTTCCTCTTTCAGCAGCTTACCTGCTGCTTTTGCTTCGCTTACGCCTTTCTCAGACAGATCCACGTCGTACCAACCGGTGAAACGGTTTTCTTTGTTCCACTGACTTTCGCCATGACGAACCAGAACCAGCTTAGTTACAGCCAT
Length:  853
Found gene gpmA -100:300 position:
GATAAGTCGCGCAGCGTCGCATCAGGCAATGTGCTCC

In [3]:
## Este módulo se utiliza para extraer y guardar secuencias de genes
# Importamos clases adicionales de Biopython para manejar secuencias y registros
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
# Función para guardar una secuencia en formato FASTA (formato estándar en bioinformática)
# Similar a guardar datos en un formato JSON o XML específico para biología
def save_as_fasta(seq, gene_name, start, end, filename):
    # Crea un objeto SeqRecord con la secuencia y metadatos (como crear un objeto con propiedades)
    fasta_record = SeqRecord(Seq(seq.upper()), id=gene_name, description=f"{gene_name} gene from position {start} to {end}")
    # Escribe el registro en un archivo FASTA
    with open(filename, "w") as output_handle:
        SeqIO.write(fasta_record, output_handle, "fasta")
    print(f"FASTA file saved as {filename}")

# Guarda la secuencia del gen extraído anteriormente en un archivo FASTA
save_as_fasta(gene_seq, gene_name, gene_position[0], gene_position[1], f"{gene_name}.fasta")

FASTA file saved as gpmA.fasta


In [4]:
## Este módulo se utiliza para extraer y preservar secuencias del genoma completo
# Función para guardar el genoma completo en formato FASTA
# Equivalente a exportar una base de datos completa a un archivo
def save_genome_as_fasta(record, filename):
    # Extrae la secuencia completa del genoma
    genome_seq = record.seq
    # Crea un objeto SeqRecord con la secuencia completa y metadatos
    genome_record = SeqRecord(genome_seq, id=record.id, description="E. coli MG1655 complete genome")
    # Escribe el registro en un archivo FASTA
    with open(filename, "w") as output_handle:
        SeqIO.write(genome_record, output_handle, "fasta")
    print(f"FASTA file of the complete genome saved as {filename}")
# Guarda el genoma completo en un archivo FASTA
save_genome_as_fasta(record, "ecoli_mg1655_genome.fasta")

FASTA file of the complete genome saved as ecoli_mg1655_genome.fasta


In [5]:
## Este módulo se utiliza para calcular la longitud promedio de los genes extraídos
# Definimos la lista de genes a procesar
genes_to_extract =['gpmA', 'fbaA', 'pykF', 'pfkB', 'pykA', 'gpmM', 'pfkA', 'fbaB']
# Inicializamos un contador para la longitud total (acumulador)
gene_long = 0
# Iteramos a través de todos los genes en la lista
for gene_name in genes_to_extract:
    # Buscamos la posición de cada gen
    gene_position = find_gene_position(record, gene_name)
    if gene_position is not None:
        # Extrae la secuencia y suma su longitud al contador
        gene_seq = record.seq[gene_position[0]-100:gene_position[1]].lower()
        gene_long += len(gene_seq)
# Imprime la longitud total de todos los genes
print(gene_long)
# Imprime la longitud promedio (dividiendo por el número de genes)
# Esto puede ser útil para estimar recursos computacionales necesarios
print(gene_long/8)

9980
1247.5
